In [1]:
import torch as t
from data import ChallengeDataset
from trainer import Trainer
from matplotlib import pyplot as plt
import numpy as np
import model
import pandas as pd
from sklearn.model_selection import train_test_split


# load the data from the csv file and perform a train-test-split
# this can be accomplished using the already imported pandas and sklearn.model_selection modules
# TODO
csv_path = 'data.csv'
tab = pd.read_csv(csv_path, sep=';')
rand = np.random.randint(1, 1000)
train_tab, test_tab = train_test_split(tab, test_size=0.2, random_state=rand)

# set up data loading for the training and validation set each using t.utils.data.DataLoader and ChallengeDataset objects
# TODO
train_dl = t.utils.data.DataLoader(ChallengeDataset(train_tab, 'train'), batch_size=10)
val_dl = t.utils.data.DataLoader(ChallengeDataset(test_tab, 'val'), batch_size=10)

/home/samil/miniconda3/envs/dl/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set up a suitable loss criterion (you can find a pre-implemented loss functions in t.nn)
# set up the optimizer (see t.optim)
# create an object of type Trainer and set its early stopping criterion
# TODO
model = model.ResNet()
crit = t.nn.BCELoss()
optimizer = t.optim.Adam(model.parameters(), lr=0.001)
trainer = Trainer(model, crit, optimizer, train_dl, val_dl, cuda=True)

In [15]:
a, b = 0, 0
for i in range(1600):
    x = ChallengeDataset(train_tab, 'train')[i][1]
    a += x[0]
    b += x[1]
a, b

(353, 95)

In [4]:
trainer.restore_checkpoint(0, '../1000/checkpoints')

In [6]:
score = 100
i = 0
while score > 60:
    i += 1
    rand = np.random.randint(1, 1000)
    train_tab, test_tab = train_test_split(tab, test_size=0.99, random_state=rand)
    train_dl = t.utils.data.DataLoader(ChallengeDataset(train_tab, 'train'), batch_size=10)
    val_dl = t.utils.data.DataLoader(ChallengeDataset(test_tab, 'val'), batch_size=10)
    trainer = Trainer(model, crit, optimizer, train_dl, val_dl, cuda=True)
    trainer.restore_checkpoint(0)
    score = float(100*trainer.val_test()[1])
    print(i, ":", "{:.2f}%".format(score))
print(rand)

1 : 70.75%


Exception ignored in: <function Format._BaseReaderWriter.__del__ at 0x7f8537fef2f0>
Traceback (most recent call last):
  File "/home/samil/miniconda3/envs/dl/lib/python3.7/site-packages/imageio/core/format.py", line 338, in __del__
    self.close()
  File "/home/samil/miniconda3/envs/dl/lib/python3.7/site-packages/imageio/core/format.py", line 351, in close
    self.request.finish()
  File "/home/samil/miniconda3/envs/dl/lib/python3.7/site-packages/imageio/core/request.py", line 571, in finish
    self._file.close()
KeyboardInterrupt: 


KeyboardInterrupt: 

In [7]:
trainer.save_onnx('last1000.onnx')

In [15]:
# # go, go, go... call fit on trainer
# res = trainer.fit(epochs=100)

In [16]:
# plot the results
# plt.plot(np.arange(len(res[0])), res[0], label='train loss')
# plt.plot(np.arange(len(res[1])), res[1], label='val loss')
# plt.yscale('log')
# plt.legend()
# plt.savefig('losses.png')